<a href="https://colab.research.google.com/github/dujest/yolov5-workshop/blob/main/Yolov5_DeepSort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount your drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/drive/MyDrive/colab

In [ ]:
%cd /content/drive/MyDrive/colab

/content/drive/MyDrive/colab


# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
# !git clone --recurse-submodules https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet.git  # clone repo
%cd /content/drive/MyDrive/colab/Yolov5_StrongSORT_OSNet
!pip install -qr requirements.txt  # install dependencies
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.12.1+cu113 (Tesla T4)


In [ ]:
%cd ../yolov5
!pip install -r requirements.txt
clear_output()
%cd ../

/content/drive/MyDrive/colab


# Download youtube video and cut



In [ ]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 3.4 MB/s 


In [ ]:
!mkdir ./videos

In [ ]:
from pytube import YouTube
import os

def downloadYouTube(videourl, path):

    yt = YouTube(videourl)
    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    if not os.path.exists(path):
        os.makedirs(path)
    yt.download(path)

In [ ]:
downloadYouTube('https://www.youtube.com/watch?v=81SJn_KuNUc', '/content/drive/MyDrive/colab/videos')

In [ ]:
# extract seconds worth of video frames of it
!yes | ffmpeg -ss 00:00:00 -i "/content/drive/MyDrive/colab/videos/Cruise port of Corfu Greece.mp4" -t 25 -c copy /content/drive/MyDrive/colab/videos/cruise_ship.avi
clear_output()

In [ ]:
!ffmpeg -i "/content/drive/MyDrive/colab/videos/cruise_ship.avi" /content/drive/MyDrive/colab/videos/cruise_ship.mp4
clear_output()

# Run inference on video

The ``cv2.imshow()`` and ``cv.imshow()`` functions from the [opencv-python](https://github.com/skvark/opencv-python) package are incompatible with Jupyter notebook; see https://github.com/jupyter/notebook/issues/3935. 

Hence we chose to save it to file in this notebook. Locally you can use the ``--show-vid`` flag in order visualize the tracking in real-time.

Download YOLOv5 pretrained weights

In [ ]:
!mkdir /content/drive/MyDrive/colab/weights

In [ ]:
!wget "https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5m.pt" -O /content/drive/MyDrive/colab/weights/yolov5m.pt

In [ ]:
%cd /content/drive/MyDrive/colab/Yolov5_StrongSORT_OSNet

/content/drive/MyDrive/colab/Yolov5_StrongSORT_OSNet


In [ ]:
!python track.py --yolo-weights "/content/drive/MyDrive/colab/yolov5/runs/train/exp/weights/best.pt" --strong-sort-weights osnet_x0_25_msmt17.pt --source "/content/drive/MyDrive/colab/videos/cruise_ship.mp4" --save-vid

# Show results

https://stackoverflow.com/questions/60977179/how-to-play-avi-file-in-google-colab

Convert avi to mp4

In [ ]:
!ffmpeg -i "/content/Yolov5_StrongSORT_OSNet/runs/track/exp/road.mp4" ../output.mp4
clear_output()

Get the file content into data_url

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('../output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

Display it with HTML

In [ ]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)